# PALTAC様 日用消費財 AI エージェント ハンズオン

このハンズオンでは、Snowflake の AI 機能を使って**商品マスタ、POSデータ、在庫データ、気象データ、ウェブサーチ**の整備から **「自然言語で質問できる AI エージェント作成」** までを一気通貫で構築します。

### 今日つくるもの

日用消費財（制汗剤・日焼け止め・カイロなど）の **売上・在庫・気象データ** を横断的に分析できる AI エージェントです。  
完成すると、例えばこんな質問に自然言語で答えてくれます：

> 「2025年の制汗剤の売上推移を月別で可視化して。気温との相関も見たい」  
> 「現在欠品している商品と、その過去の売上トレンドを教えて」  
> 「花王の商品一覧とそれぞれの特徴を教えて」
> 「2026年の花王の新商品は何？」

---

### ハンズオンの流れ（所要時間：約80分）

| Step | やること | 学べる Snowflake 機能 | 所要時間 |
|:----:|----------|----------------------|:--------:|
| 1 | AI で商品マスタを整備 | AI_CLASSIFY / AI_COMPLETE | 10分 |
| 2 | 商品の意味検索を可能にする | Cortex Search Service | 10分 |
| 3 | 気象データを取り込む | Snowflake Marketplace | 10分 |
| 4 | データの意味を定義する | Semantic View（Cortex Analyst） | 20分 |
| 5 | AI エージェントを組み立てる | Cortex Agent | 15分 |
| - | 動かしてみよう！ | - | 5分 |


## Step 0: 事前準備
使用するデータベース・スキーマ・ウェアハウスを設定します。

In [ ]:
USE DATABASE PALTAC_HANDSON;
USE SCHEMA ANALYTICS;
USE WAREHOUSE COMPUTE_WH;

---
## Step 1: AI で商品マスタを整備する

現在の商品マスタ（`RAW.PRODUCT_MASTER`）には、JANコード・商品名・メーカー名しかありません。  
ここに Snowflake の **AI SQL 関数** を使って、2つの情報を自動で追加します。

### 追加する情報

#### 1. 大カテゴリの自動分類 — `AI_CLASSIFY`

商品名だけを見て、AI が自動的にカテゴリを判定します。  
あらかじめ定義した **5つのカテゴリ** に分類します（MECE＝漏れなくダブりなく）。

| カテゴリ | どんな商品？ |
|----------|-------------|
| ビューティーケア | 化粧品、スキンケア、日焼け止め |
| パーソナルケア | シャンプー、ボディソープ、制汗剤、リップ |
| ヘルスケア | 目薬、冷却シート、カイロ、衛生用品 |
| ホームケア | 洗濯洗剤、芳香剤、消臭剤、防虫剤 |
| 日用雑貨 | ティッシュ、紙製品、文具、電池 |

#### 2. 商品キャプションの自動生成 — `AI_COMPLETE`

商品名とメーカー名から、その商品の **魅力や特徴を伝える説明文（約300文字）** を AI が自動で生成します。

> **ポイント**: `AI_CLASSIFY` はラベル＋説明文を渡すだけで分類してくれるので、従来のルールベースのロジックを書く必要がありません。

In [ ]:
SELECT * FROM PALTAC_HANDSON.RAW.PRODUCT_MASTER;

それでは実行しましょう。商品数にもよりますが、**約1〜2分** で完了します。

In [ ]:
CREATE OR REPLACE TABLE PALTAC_HANDSON.ANALYTICS.AI_PRODUCT_MASTER AS
SELECT 
    JANCODE,
    PRODUCT_NAME,
    MAKER_NAME,
    MAKER_CODE,
    -- ① AI_CLASSIFY関数で大カテゴリを分類（MECE）
    AI_CLASSIFY(
        PRODUCT_NAME,
        [
            {'label': 'ビューティーケア', 'description': '化粧品、スキンケア、メイクアップ、日焼け止め'},
            {'label': 'パーソナルケア', 'description': 'シャンプー、ボディソープ、オーラルケア、制汗剤、ハンドケア、リップケア'},
            {'label': 'ヘルスケア', 'description': 'OTC医薬品、目薬、サプリメント、冷却シート、カイロ、衛生用品'},
            {'label': 'ホームケア', 'description': '洗濯洗剤、住居用洗剤、芳香剤、消臭剤、除湿剤、防虫剤'},
            {'label': '日用雑貨', 'description': 'ティッシュ、紙製品、収納用品、文具、電池'}
        ],
        {'task_description': '日用消費財の商品名から最も適切な大カテゴリに分類してください'}
    ):labels[0]::VARCHAR AS CATEGORY,
    -- ② AI_COMPLETE関数で商品キャプション生成
    TRIM(SNOWFLAKE.CORTEX.AI_COMPLETE(
        'llama3.1-405b',
        '以下の商品の魅力や特徴を伝えるキャプション（300文字程度）を日本語で1つ作成してください。
         商品名: ' || PRODUCT_NAME || '
         メーカー: ' || COALESCE(MAKER_NAME, '不明') || '
         キャプション:'
    )) AS PRODUCT_CAPTION,
    CURRENT_TIMESTAMP() AS CREATED_AT
FROM PALTAC_HANDSON.RAW.PRODUCT_MASTER;

### 結果を見てみましょう

AI が生成した `CATEGORY`（カテゴリ）と `PRODUCT_CAPTION`（商品キャプション）を確認します。

In [ ]:
SELECT * FROM PALTAC_HANDSON.ANALYTICS.AI_PRODUCT_MASTER;

### カテゴリ分類の精度を確認

5つのカテゴリにバランスよく分類されているか確認しましょう。  
偏りがある場合は、`AI_CLASSIFY` に渡すカテゴリの `description` を調整することで精度を改善できます。

In [ ]:
SELECT 
    CATEGORY,
    COUNT(*) AS PRODUCT_COUNT
FROM PALTAC_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
GROUP BY CATEGORY
ORDER BY PRODUCT_COUNT DESC;

## Step 2: 気象データを取り込む（Snowflake Marketplace）

日用消費財の売上は天候に大きく左右されます。  
カイロは気温が下がると売れ、制汗剤は暑くなると売れる — こうした **天候×売上の分析** を可能にするため、  
Snowflake Marketplace の **気象データ** を取り込みます。

### データソース

**Weather Source LLC: Frostbyte** が提供するグローバル気象データを使用します。  
日本（JP）の日次気象データを 2019年〜 取得できます。

### なぜテーブルを作るのか？

元データは華氏・インチ単位で、グローバルデータが含まれています。  
POSデータと結合するには **日本のデータのみを抽出し、メートル法に変換して日付ごとに1行** にする必要があるため、  
全国平均に集約するテーブルを作成します。

| 集約カラム | 計算方法 | 説明 |
|------------|----------|------|
| 平均気温 | AVG | 全国の観測所の平均（℃に変換） |
| 最高気温 | MAX | 全国で最も高かった気温（℃に変換） |
| 最低気温 | MIN | 全国で最も低かった気温（℃に変換） |
| 降水量 | AVG | 全国平均の降水量（mmに変換） |
| 湿度 | AVG | 全国平均の湿度 |
| 日照時間 | AVG | 全国平均の日射量 |
| 降雪量 | SUM | 全国の降雪量合計（cmに変換） |
| 風速 | AVG | 全国平均の風速（km/hに変換） |

In [ ]:
CREATE OR REPLACE TABLE PALTAC_HANDSON.ANALYTICS.DAILY_WEATHER AS
SELECT
    DATE_VALID_STD                                                      AS WEATHER_DATE,
    ROUND(AVG((AVG_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS AVG_TEMPERATURE,
    ROUND(MAX((MAX_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MAX_TEMPERATURE,
    ROUND(MIN((MIN_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MIN_TEMPERATURE,
    ROUND(AVG(TOT_PRECIPITATION_IN * 25.4), 1)                          AS AVG_RAINFALL,
    ROUND(AVG(AVG_HUMIDITY_RELATIVE_2M_PCT), 1)                         AS AVG_HUMIDITY,
    ROUND(AVG(AVG_RADIATION_SOLAR_TOTAL_WPM2), 2)                       AS AVG_SUNLIGHT_HOURS,
    ROUND(SUM(TOT_SNOWFALL_IN * 25.4), 1)                               AS TOTAL_SNOWFALL,
    ROUND(AVG(AVG_WIND_SPEED_10M_MPH * 1.60934), 1)                     AS AVG_WIND_SPEED
FROM WEATHER_SOURCE_LLC_FROSTBYTE.ONPOINT_ID.HISTORY_DAY
WHERE COUNTRY = 'JP'
GROUP BY DATE_VALID_STD
ORDER BY DATE_VALID_STD;

In [ ]:
SELECT * FROM PALTAC_HANDSON.ANALYTICS.DAILY_WEATHER;

---
## Step 3: 商品の意味検索を可能にする（Cortex Search）

Step 1 で作った `AI_PRODUCT_MASTER` に対して、**Cortex Search Service** を作成します。

### Cortex Search とは？

テキストの **意味** を理解して検索するサービスです。  
キーワードの完全一致ではなく、「こういう意味のものを探して」という検索ができます。

例えば「夏に使える涼しくなる商品」と聞くと、「冷却シート」や「制汗スプレー」がヒットします。

### 仕組み

商品名・メーカー名・カテゴリ・AIキャプションを **1つのテキストに結合** し、  
Snowflake の Embedding モデル（`snowflake-arctic-embed-l-v2.0`）でベクトル化して検索可能にします。

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE PALTAC_HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES JANCODE, PRODUCT_NAME, MAKER_NAME, CATEGORY, PRODUCT_CAPTION
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        JANCODE,
        PRODUCT_NAME,
        MAKER_NAME,
        MAKER_CODE,
        CATEGORY,
        PRODUCT_CAPTION,
        CONCAT(
            COALESCE(PRODUCT_NAME, ''), ' ',
            COALESCE(MAKER_NAME, ''), ' ',
            COALESCE(CATEGORY, ''), ' ',
            COALESCE(PRODUCT_CAPTION, '')
        ) AS SEARCH_TEXT
    FROM PALTAC_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
);

---
## Step 4: データの意味を定義する（セマンティックビュー）

ここがこのハンズオンの **キモ** です。

Cortex Analyst に「売上を教えて」と聞いたとき、どのテーブルの・どのカラムを・どう集計すればよいかを  
AI が理解できるように、**データの意味（セマンティクス）** を定義します。

### セマンティックビューで定義すること

| 定義項目 | 何を定義する？ | 具体例 |
|----------|---------------|--------|
| **テーブル** | 分析対象のテーブルとその説明 | POS_DATA, INVENTORY, WEATHER |
| **リレーションシップ** | テーブル間の結合条件 | 売上日 = 気象観測日 |
| **ファクト** | 集計対象の数値カラム | 販売数量、売上金額、気温 |
| **ディメンション** | 分析の切り口となる属性 | 商品名、メーカー、日付 |
| **メトリクス** | よく使う集計パターン | 総売上金額＝SUM(売上金額) |
| **シノニム** | 日本語の別名・同義語 | 「売上」「販売」「POS」 |

### 3つのテーブルの関係

```
           JANCODE              SALE_DATE = WEATHER_DATE
POS_DATA ──────────▶ INVENTORY    POS_DATA ──────────────────▶ WEATHER
(売上)                (在庫)       (売上)                       (気象)
```

> **ポイント**: シノニムを豊富に定義しておくと、「売上」でも「販売」でも「POS」でも同じテーブルを参照してくれます。

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW PALTAC_ANALYST

    -- ========================================
    -- テーブル定義
    -- ========================================
    tables (
        POS_DATA AS PALTAC_HANDSON.ANALYTICS.POS_DATA
            PRIMARY KEY (JANCODE, SALE_DATE)
            WITH SYNONYMS = ('売上', '販売', '売上データ', '販売データ', 'POS', 'POSデータ', '売上実績')
            COMMENT = '日次POSデータ。2022-01-01〜2026-02-24の商品別日次販売数量・売上金額を含む。',

        INVENTORY AS PALTAC_HANDSON.ANALYTICS.INVENTORY
            PRIMARY KEY (JANCODE)
            WITH SYNONYMS = ('在庫', '在庫データ', '在庫状況', '在庫残', '棚卸', '在庫スナップショット')
            COMMENT = '在庫スナップショット（2026-02-24時点）。商品別の在庫数量・在庫金額・在庫ステータスを含む。',

        WEATHER AS PALTAC_HANDSON.ANALYTICS.DAILY_WEATHER
            PRIMARY KEY (WEATHER_DATE)
            WITH SYNONYMS = ('気象', '天気', '気象データ', '天気データ', '気温', '降水量', '天候', '気象庁', '気象情報')
            COMMENT = '全国日次平均気象データ（2015年〜）。気象庁の全観測所データを日別に集約した全国平均の気温・降水量・湿度・日照時間・降雪量・風速。'
    )

    -- ========================================
    -- リレーションシップ定義
    -- ========================================
    relationships (
        POS_INVENTORY AS POS_DATA(JANCODE) REFERENCES INVENTORY(JANCODE),
        POS_WEATHER AS POS_DATA(SALE_DATE) REFERENCES WEATHER(WEATHER_DATE)
    )

    -- ========================================
    -- ファクト定義（数値データ）
    -- ========================================
    facts (
        -- 売上ファクト
        POS_DATA.sales_qty_fact AS SALES_QTY
            COMMENT = '日次販売数量（個）',
        POS_DATA.unit_price_fact AS UNIT_PRICE
            COMMENT = '販売単価（円）',
        POS_DATA.sales_amount_fact AS SALES_AMOUNT
            COMMENT = '日次売上金額（円）。販売数量×単価。',

        -- 在庫ファクト
        INVENTORY.stock_qty_fact AS STOCK_QTY
            COMMENT = '在庫数量（個）',
        INVENTORY.inv_unit_price_fact AS UNIT_PRICE
            COMMENT = '在庫単価（円）',
        INVENTORY.stock_amount_fact AS STOCK_AMOUNT
            COMMENT = '在庫金額（円）。在庫数量×単価。',

        -- 気象ファクト
        WEATHER.avg_temperature_fact AS AVG_TEMPERATURE
            COMMENT = '全国日平均気温（℃）。全観測所の平均。',
        WEATHER.max_temperature_fact AS MAX_TEMPERATURE
            COMMENT = '全国日最高気温（℃）。全観測所の最大値。',
        WEATHER.min_temperature_fact AS MIN_TEMPERATURE
            COMMENT = '全国日最低気温（℃）。全観測所の最小値。',
        WEATHER.avg_rainfall_fact AS AVG_RAINFALL
            COMMENT = '全国日平均降水量（mm）',
        WEATHER.avg_humidity_fact AS AVG_HUMIDITY
            COMMENT = '全国日平均湿度（%）',
        WEATHER.avg_sunlight_hours_fact AS AVG_SUNLIGHT_HOURS
            COMMENT = '全国日平均日照時間（時間）',
        WEATHER.total_snowfall_fact AS TOTAL_SNOWFALL
            COMMENT = '全国日降雪量合計（cm）',
        WEATHER.avg_wind_speed_fact AS AVG_WIND_SPEED
            COMMENT = '全国日平均風速（m/s）'
    )

    -- ========================================
    -- ディメンション定義（属性データ）
    -- ========================================
    dimensions (
        -- ========== POS_DATA ディメンション ==========
        POS_DATA.jancode AS JANCODE
            WITH SYNONYMS = ('JANコード', '商品コード', '製品コード', 'バーコード', 'JAN')
            COMMENT = '商品識別用のJANコード（13桁）',
        POS_DATA.product_name AS PRODUCT_NAME
            WITH SYNONYMS = ('商品名', '製品名', '品名', '名前')
            COMMENT = '日用消費財の商品名。制汗剤、日焼け止め、カイロ、ハンドソープなどを含む。',
        POS_DATA.maker_name AS MAKER_NAME
            WITH SYNONYMS = ('メーカー', 'メーカー名', '製造元', 'ブランド', 'メーカ')
            COMMENT = '商品の製造メーカー名（ライオン、小林製薬、ロート製薬、花王など）',
        POS_DATA.sale_date AS SALE_DATE
            WITH SYNONYMS = ('売上日', '販売日', '取引日', '日付')
            COMMENT = '売上が発生した日付（DATE型、2022-01-01〜2026-02-24）',
        POS_DATA.sale_year AS YEAR(SALE_DATE)
            WITH SYNONYMS = ('売上年', '販売年', '年', '対象年')
            COMMENT = '売上が発生した年（2022〜2026）',
        POS_DATA.sale_month AS MONTH(SALE_DATE)
            WITH SYNONYMS = ('売上月', '販売月', '月')
            COMMENT = '売上が発生した月（1〜12）',

        -- ========== INVENTORY ディメンション ==========
        INVENTORY.jancode AS JANCODE
            WITH SYNONYMS = ('JANコード', '商品コード', '製品コード', 'バーコード')
            COMMENT = '商品識別用のJANコード',
        INVENTORY.product_name AS PRODUCT_NAME
            WITH SYNONYMS = ('商品名', '製品名', '品名')
            COMMENT = '商品名',
        INVENTORY.maker_name AS MAKER_NAME
            WITH SYNONYMS = ('メーカー', 'メーカー名', '製造元')
            COMMENT = 'メーカー名',
        INVENTORY.snapshot_date AS SNAPSHOT_DATE
            WITH SYNONYMS = ('在庫日', '棚卸日', '基準日', 'スナップショット日')
            COMMENT = '在庫データの取得日（2026-02-24）',
        INVENTORY.stock_status AS STOCK_STATUS
            WITH SYNONYMS = ('在庫ステータス', '在庫状態', '在庫区分', 'ステータス')
            COMMENT = '在庫状態を表すステータス（通常 / 残少 / 欠品）',

        -- ========== WEATHER ディメンション ==========
        WEATHER.weather_date AS WEATHER_DATE
            WITH SYNONYMS = ('観測日', '気象日', '天気の日付', '気象観測日')
            COMMENT = '気象データの観測日（DATE型、2015年〜）。POS_DATAのSALE_DATEとリレーションで紐づく。',
        WEATHER.weather_year AS YEAR(WEATHER_DATE)
            WITH SYNONYMS = ('観測年', '気象年')
            COMMENT = '気象データの観測年',
        WEATHER.weather_month AS MONTH(WEATHER_DATE)
            WITH SYNONYMS = ('観測月', '気象月')
            COMMENT = '気象データの観測月（1〜12）'
    )

    -- ========================================
    -- メトリクス定義（集計関数）
    -- ========================================
    metrics (
        -- 売上メトリクス
        POS_DATA.total_sales_amount AS SUM(POS_DATA.sales_amount_fact)
            WITH SYNONYMS = ('総売上金額', '合計売上金額', '売上合計', '売上総額')
            COMMENT = '売上金額の合計（円）',
        POS_DATA.total_sales_qty AS SUM(POS_DATA.sales_qty_fact)
            WITH SYNONYMS = ('総販売数量', '合計販売数量', '販売数合計', '総販売数')
            COMMENT = '販売数量の合計（個）',
        POS_DATA.average_unit_price AS AVG(POS_DATA.unit_price_fact)
            WITH SYNONYMS = ('平均単価', '平均販売単価', '平均売価')
            COMMENT = '販売単価の平均（円）',
        POS_DATA.average_daily_sales AS AVG(POS_DATA.sales_amount_fact)
            WITH SYNONYMS = ('日平均売上', '日次平均売上', '平均日次売上')
            COMMENT = '日次売上金額の平均（円）',

        -- 在庫メトリクス
        INVENTORY.total_stock_qty AS SUM(INVENTORY.stock_qty_fact)
            WITH SYNONYMS = ('総在庫数量', '合計在庫数量', '在庫数合計', '在庫総数')
            COMMENT = '在庫数量の合計（個）',
        INVENTORY.total_stock_amount AS SUM(INVENTORY.stock_amount_fact)
            WITH SYNONYMS = ('総在庫金額', '合計在庫金額', '在庫金額合計', '在庫総額')
            COMMENT = '在庫金額の合計（円）',
        INVENTORY.average_stock_price AS AVG(INVENTORY.inv_unit_price_fact)
            WITH SYNONYMS = ('平均在庫単価', '平均原価')
            COMMENT = '在庫単価の平均（円）',

        -- 気象メトリクス
        WEATHER.period_avg_temperature AS AVG(WEATHER.avg_temperature_fact)
            WITH SYNONYMS = ('平均気温', '気温平均', '平均温度')
            COMMENT = '期間内の全国平均気温（℃）',
        WEATHER.period_max_temperature AS MAX(WEATHER.max_temperature_fact)
            WITH SYNONYMS = ('最高気温', '最高温度', '期間最高気温')
            COMMENT = '期間中の全国最高気温（℃）',
        WEATHER.period_min_temperature AS MIN(WEATHER.min_temperature_fact)
            WITH SYNONYMS = ('最低気温', '最低温度', '期間最低気温')
            COMMENT = '期間中の全国最低気温（℃）',
        WEATHER.period_avg_rainfall AS AVG(WEATHER.avg_rainfall_fact)
            WITH SYNONYMS = ('平均降水量', '降水量平均')
            COMMENT = '期間内の全国平均降水量（mm）',
        WEATHER.period_avg_humidity AS AVG(WEATHER.avg_humidity_fact)
            WITH SYNONYMS = ('平均湿度', '湿度平均')
            COMMENT = '期間内の全国平均湿度（%）',
        WEATHER.period_avg_sunlight AS AVG(WEATHER.avg_sunlight_hours_fact)
            WITH SYNONYMS = ('平均日照時間', '日照時間平均')
            COMMENT = '期間内の全国平均日照時間（時間）',
        WEATHER.period_total_snowfall AS SUM(WEATHER.total_snowfall_fact)
            WITH SYNONYMS = ('総降雪量', '合計降雪量', '累積降雪量')
            COMMENT = '期間内の全国降雪量合計（cm）',
        WEATHER.period_avg_wind_speed AS AVG(WEATHER.avg_wind_speed_fact)
            WITH SYNONYMS = ('平均風速', '風速平均')
            COMMENT = '期間内の全国平均風速（m/s）'
    )

    COMMENT = 'PALTAC日用消費財の売上・在庫・気象分析のためのセマンティックビュー。制汗剤、日焼け止め、カイロ、冷却シートなどの季節商品を含む。2022年〜の日次POSデータ、2026-02-24時点の在庫スナップショット、および気象庁の全国日次平均気象データ（気温・降水量・湿度・日照時間・降雪量等）を統合。POS売上日と気象観測日がリレーションで紐づき、天候と売上の相関分析が可能。';

### セマンティックビューの確認

正しく定義できたか確認しましょう。テーブル・ファクト・ディメンション・メトリクスの一覧が表示されます。

In [ ]:
DESCRIBE SEMANTIC VIEW PALTAC_ANALYST;

---
## Step 5: AI エージェントを組み立てる（Cortex Agent）

いよいよ最後のステップです。  
ここまで作ってきた **Cortex Search** と **Cortex Analyst** を組み合わせて、  
1つの **AI エージェント** にまとめます。

### エージェントの仕組み

```
ユーザーの質問
     │
     ▼
┌─────────────────────────┐
│    PALTAC Agent        │
│   （オーケストレーター）   │
│                        │
│  質問の意図を判断して      │
│  適切なツールを選択        │
└────┬──────────┬─────────┘
     │          │
     ▼          ▼
┌─────────┐ ┌──────────────┐
│ Cortex  │ │   Cortex     │
│ Search  │ │  Analyst     │
│         │ │              │
│商品情報の│  │売上・在庫・    │
│意味検索  │ │気象の数値分析   │
└─────────┘ └──────────────┘
```

### ツールの使い分け

| 質問の種類 | 使われるツール | 例 |
|-----------|---------------|----|
| 商品の特徴・情報を知りたい | **Cortex Search** | 「花王の商品を教えて」 |
| 数値の集計・分析をしたい | **Cortex Analyst** | 「2025年の月別売上は？」 |
| 両方が必要 | **Search → Analyst** | 「欠品商品の売上トレンドは？」 |

In [ ]:
CREATE OR REPLACE AGENT PALTAC_HANDSON.ANALYTICS.PALTAC_AGENT
    COMMENT = 'PALTAC日用消費財の売上・在庫・気象分析と商品検索を行うエージェント'
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたはPALTAC（日用消費財卸）の商品データアナリストです。ユーザーからの質問に対して、売上データ（POS）、在庫データ、商品マスタ、気象データを活用して回答してください。数値データは適切にフォーマットして表示してください（例：金額は円表記・カンマ区切り、数量は個数表記）。商品名は正確に記載してください。分析結果には根拠となるデータを含めてください。日本語で丁寧に回答してください。可能であれば視覚化を提供してください。時系列のトレンドは線グラフをデフォルトとし、カテゴリ比較は棒グラフを使用してください。季節商品（制汗剤、日焼け止め、カイロ、冷却シートなど）の分析では、気象データとの相関も考慮してください。",
    "orchestration": "商品の詳細情報（商品名、メーカー、カテゴリ、AIキャプションなど）を検索する場合はcortex searchを使用し、売上・在庫・気象の数値分析にはcortex analystを使用してください。複合的な質問の場合は、まずcortex searchで商品を特定し、その結果をcortex analystに渡して分析してください。気温や天候と売上の関係を聞かれた場合は、cortex analystでPOSデータと気象データを組み合わせて分析してください。",
    "sample_questions": [
      {
        "question": "2025年の制汗剤の売上推移を月別で可視化してください。気温との相関も見たいです。"
      },
      {
        "question": "カイロの売上が最も多い月はいつですか？気温が何度以下になると売上が急増しますか？"
      },
      {
        "question": "現在欠品している商品を教えてください。それぞれの商品の過去の売上トレンドも見せてください。"
      },
      {
        "question": "花王の商品一覧とそれぞれの特徴を教えてください。"
      },
      {
        "question": "2024年と2025年で売上が大きく伸びた商品トップ5を教えてください。"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "PALTAC_ANALYST",
        "description": "PALTAC日用消費財の売上データ（POS）、在庫データ、全国日次気象データをクエリして分析します。売上金額、販売数量、在庫数量、在庫金額の集計や、気温・降水量・降雪量と売上の相関分析に使用します。2022年〜2026年2月の日次売上データ、2026年2月時点の在庫スナップショット、2015年〜の全国平均気象データを含みます。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "PRODUCT_SEARCH",
        "description": "PALTAC日用消費財の商品マスタから商品情報を検索します。商品名、メーカー名、AIが分類したカテゴリ（ビューティーケア、パーソナルケア、ヘルスケア、ホームケア、日用雑貨）、AIが生成した商品キャプションなどのテキスト検索に使用します。"
      }
    }
  ],
  "tool_resources": {
    "PALTAC_ANALYST": {
      "semantic_view": "PALTAC_HANDSON.ANALYTICS.PALTAC_ANALYST"
    },
    "PRODUCT_SEARCH": {
      "name": "PALTAC_HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE",
      "max_results": 10
    }
  }
}
$$;

### エージェントの確認

In [ ]:
SHOW AGENTS IN SCHEMA PALTAC_HANDSON.ANALYTICS;

---
## 動かしてみよう！

エージェントが完成しました！Snowsight の **Cortex Agent UI** から以下のような質問を試してみましょう。

### 試してみたい質問例

**売上分析**
- 「2025年の制汗剤の売上推移を月別で可視化してください。気温との相関も見たいです。」
- 「2024年と2025年で売上が大きく伸びた商品トップ5を教えてください。」

**天候×売上**
- 「カイロの売上が最も多い月はいつですか？気温が何度以下になると売上が急増しますか？」
- 「雨の日は売上にどんな影響がありますか？」

**在庫管理**
- 「現在欠品している商品を教えてください。それぞれの過去の売上トレンドも見せてください。」
- 「在庫が残少の商品はどれですか？」

**商品検索**
- 「花王の商品一覧とそれぞれの特徴を教えてください。」
- 「夏に売れそうな商品をおすすめしてください。」

**Web検索**
- 「花王の2026年度の最新商品を教えて欲しい。」
- 「ここ数ヶ月でSNSでバズっている商品はないでしょうか。」